# Bacterial Growth: a complex continuous system

From: https://juliadynamics.github.io/AgentsExampleZoo.jl/dev/examples/growing_bacteria/

In [ ]:
using Agents
using LinearAlgebra
using Random

## Agents

Agents will be splitting into more agents, thus having agent generation in continuous space. The model also uses advanced agent movement in continuous space, where a specialized `move_agent` function is created.

In [ ]:
mutable struct SimpleCell <: AbstractAgent
    id::Int
    pos::NTuple{2, Float64}
    length::Float64
    orientation::Float64
    growthprog::Float64
    growthrate::Float64

    # node positions/forces
    p1::NTuple{2, Float64}
    p2::NTuple{2, Float64}
    f1::NTuple{2, Float64}
    f2::NTuple{2, Float64}
end

# Specialized constructor
function SimpleCell(id, pos, l, φ, g, γ)
    a = SimpleCell(id, pos, l, φ, g, γ, (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0))
    update_nodes!(a)
    return a
end 

In this model, the agents have to store their state in two redundant ways: the cell coordinates (position, length, orientation) are required for the equations of motion, while the positions of the disk-shaped nodes are necessary for calculating mechanical forces between cells. To transform from one set of coordinates to the other, we need to write a function:

In [ ]:
# geometry convenience functions
unitvector(φ) = reverse(sincos(φ))
cross2D(a, b) = a[1] * b[2] - a[2] * b[1]

function update_nodes!(a::SimpleCell)
    offset = 0.5 * a.length .* unitvector(a.orientation)
    a.p1 = a.pos .+ offset
    a.p2 = a.pos .- offset
end

## Stepping functions

In [ ]:
function model_step!(model)
    extent = model.space.extent
    for a in allagents(model)
        if a.growthprog ≥ 1
            # When a cell has matured, it divides into two daughter cells on the
            # positions of its nodes.
            add_agent!(a.p1, model, 0.0, a.orientation, 0.0, 0.1 * rand(model.rng) + 0.05)
            add_agent!(a.p2, model, 0.0, a.orientation, 0.0, 0.1 * rand(model.rng) + 0.05)
            kill_agent!(a, model)
        else
            # The rest lengh of the internal spring grows with time. This causes
            # the nodes to physically separate.
            uv = unitvector(a.orientation)
            internalforce = model.hardness * (a.length - a.growthprog) .* uv
            a.f1 = -1 .* internalforce
            a.f2 = internalforce
        end
    end
    # Bacteria can interact with more than on other cell at the same time, therefore,
    # we need to specify the option `:all` in `interacting_pairs`
    for (a1, a2) in interacting_pairs(model, 2.0, :all)
        interact!(a1, a2, model)
    end
end

In [ ]:
function agent_step!(agent::SimpleCell, model::ABM)
    fsym, compression, torque = transform_forces(agent)
    direction =  model.dt * model.mobility .* fsym
    walk!(agent, direction, model)
    agent.length += model.dt * model.mobility .* compression
    agent.orientation += model.dt * model.mobility .* torque
    agent.growthprog += model.dt * agent.growthrate
    update_nodes!(agent)
    return agent.pos
end

## Helper functions

In [ ]:
function interact!(a1::SimpleCell, a2::SimpleCell, model)
    n11 = noderepulsion(a1.p1, a2.p1, model)
    n12 = noderepulsion(a1.p1, a2.p2, model)
    n21 = noderepulsion(a1.p2, a2.p1, model)
    n22 = noderepulsion(a1.p2, a2.p2, model)
    a1.f1 = @. a1.f1 + (n11 + n12)
    a1.f2 = @. a1.f2 + (n21 + n22)
    a2.f1 = @. a2.f1 - (n11 + n21)
    a2.f2 = @. a2.f2 - (n12 + n22)
end

function noderepulsion(p1::NTuple{2,Float64}, p2::NTuple{2,Float64}, model::ABM)
    delta = p1 .- p2
    distance = norm(delta)
    if distance ≤ 1
        uv = delta ./ distance
        return (model.hardness * (1 - distance)) .* uv
    end
    return (0.0, 0.0)
end

function transform_forces(agent::SimpleCell)
    # symmetric forces (CM movement)
    fsym = agent.f1 .+ agent.f2
    # antisymmetric forces (compression, torque)
    fasym = agent.f1 .- agent.f2
    uv = unitvector(agent.orientation)
    compression = dot(uv, fasym)
    torque = 0.5 * cross2D(uv, fasym)
    return fsym, compression, torque
end

## Animating bacterial growth

In [ ]:
function initialise(; 
    spacesize=(14, 9),
    dt = 0.005,
    hardness = 100.0,
    mobility = 1.0,
    seed = 1680
)
    properties = Dict(:dt => dt, :hardness => hardness, :mobility => mobility)
    space = ContinuousSpace(spacesize, 1.0; periodic = false)
    rng = MersenneTwister(seed)
    model = ABM(
        SimpleCell,
        space;
        properties = properties,
        rng = rng
    )
end

In [ ]:
model = initialise()

In [ ]:
# Start with two bacteria
add_agent!((6.5, 4.0), model, 0.0, 0.3, 0.0, 0.1)
add_agent!((7.5, 4.0), model, 0.0, 0.0, 0.0, 0.1)

In [ ]:
using InteractiveDynamics
using CairoMakie

function cassini_oval(agent)
    t = LinRange(0, 2π, 50)
    a = agent.growthprog
    b = 1
    m = @. 2 * sqrt((b^4 - a^4) + a^4 * cos(2 * t)^2) + 2 * a^2 * cos(2 * t)
    C = sqrt.(m / 2)

    x = C .* cos.(t)
    y = C .* sin.(t)

    uv = reverse(sincos(agent.orientation))
    θ = atan(uv[2], uv[1])
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]

    bacteria = R * permutedims([x y])
    coords = [Point2f(x, y) for (x, y) in zip(bacteria[1, :], bacteria[2, :])]
    scale(Polygon(coords), 0.5)
end

In [ ]:
bacteria_color(agent) = CairoMakie.RGBf(agent.id * 3.14 % 1, 0.2, 0.2)

In [ ]:
abmvideo(
    "_static/bacteria.mp4", model, agent_step!, model_step!;
    am = cassini_oval, ac = bacteria_color,
    spf = 50, framerate = 30, frames = 300,
    title = "Growing bacteria"
)